In [ ]:
from google.colab import drive
drive.mount("/content/drive")

READ DATA

In [ ]:
import numpy as np 
import pickle 
from random import shuffle
import random
random.seed(0)


# CNN features 
features_dl =  np.load("/content/drive/My Drive/AIA Project/Features from DL recognition/DL_recognition_features", allow_pickle = True)


train_features = np.zeros((1120,2240))
test_features = np.zeros((1120,2240))
train_labels = []
test_labels = []

for i in range(224):
  ind = [j for j in range(10)]
  shuffle(ind)
  for k in range(5):
    train_features[5*i+k]= features_dl[10*i+ind[k]]
    test_features[5*i+k] = features_dl[10*i+ind[k+5]]
    train_labels.append(i)
    test_labels.append(i)
  
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)







#extracted features from DL polar images 
train_features = np.load("/content/drive/My Drive/DataShuffled/feature_vector_Train_DL", allow_pickle = True)
train_labels = np.load("/content/drive/My Drive/DataShuffled/train_labels_polar_dl", allow_pickle = True)
test_features = np.load("/content/drive/My Drive/DataShuffled/feature_vector_Test_DL", allow_pickle = True)
test_labels = np.load("/content/drive/My Drive/DataShuffled/test_labels_polar_dl", allow_pickle = True)




#extracted features from IP polar images:
train_features = np.load("/content/drive/My Drive/feature_vector_Train_25", allow_pickle = True)
test_labels = np.load("/content/drive/My Drive/DataShuffled/training_subjects", allow_pickle = True)
test_features = np.load("/content/drive/My Drive/feature_vector_Test_25", allow_pickle = True)
train_labels = np.load("/content/drive/My Drive/DataShuffled/test_subjects", allow_pickle = True)

In [ ]:
# Shuffle train_data with labels : 

from random import shuffle
import random 

ind_list = [i for i in range(1120)]
random.seed(0)
shuffle(ind_list)
train_features  = train_features[ind_list]
y_train= train_labels[ind_list]



PCA : 99% for DL/IP polar images, 90% for CNN features

In [ ]:
#Apply PCA :
from sklearn.decomposition import PCA 

pca = PCA(.99, whiten= True)
pca.fit(train_features)
train_features_pca = pca.transform(train_features)
test_features_pca = pca.transform(test_features)


print(train_features_pca.shape, test_features_pca.shape)

In [ ]:
# function to transform labels to one hot (to use AUC) :

def one_hot(labels):
  one_hot = []
  for i in range(len(labels)):
    y = np.zeros((224))
    y[labels[i]]=1
    one_hot.append(y)
  one_hot = np.array(one_hot)
  return one_hot

ML models

K Nearest Neighbours

In [ ]:
import sklearn
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score



knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)
knn.fit(train_features_pca, y_train)

scores = knn.predict(train_features_pca)

# Show train accuracy
print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(y_train, scores)))

y_pred = knn.predict(test_features_pca)

# Show prediction accuracy
print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(test_labels, y_pred)))
print('\nPrecision:')
print('{:.2%}\n'.format(precision_score(test_labels, y_pred,average='weighted')))
print('\nrecall:')
print('{:.2%}\n'.format(recall_score(test_labels, y_pred,average='weighted')))
print('\nf1 score:')
print('{:.2%}\n'.format(f1_score(test_labels, y_pred,average='weighted')))
print('\n auc:')
print('{:.2%}\n'.format(roc_auc_score(one_hot(test_labels), one_hot(y_pred), average='weighted', multi_class='ovo')))

SVM 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
#Let's apply a stratified 10-fold cross validation

#Let's build a pipeline with only a scaler and a linear SVM
pipe = Pipeline([('scaler', StandardScaler()),('classifier',SVC())])

#Let's build a structure for the grid search with a set of parameters for the techniques in the pipeline
#Be careful to the double underscore between the name you chose in the pipeline and the name of the parameter
#Be careful also to the function logspace for the range of the C values
params = dict()
params['classifier__C'] = np.logspace(-5, 5, 10)
params['classifier__kernel'] = ['linear', 'poly', 'rbf']
params['classifier__gamma'] = [0.01, 0.1, 1]

#Let's apply the grid search function
grid_search_svm = GridSearchCV(pipe, params)
#Let's train and test the learning system
grid_search_svm.fit(train_features_pca, y_train)

#Here we just print the best accuracy and the corresponding values for the parameters
print("The best parameters are %s with an accuracy of %0.4f"%(grid_search_svm.best_params_, grid_search_svm.best_score_))


y_pred= grid_search_svm.predict(test_features_pca)
cm = confusion_matrix(y_pred, test_labels)

# Svm metrics 

print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(test_labels, y_pred)))
print('\nPrecision:')
print('{:.2%}\n'.format(precision_score(test_labels, y_pred,average='weighted')))
print('\nrecall:')
print('{:.2%}\n'.format(recall_score(test_labels, y_pred,average='weighted')))
print('\nf1 score:')
print('{:.2%}\n'.format(f1_score(test_labels, y_pred,average='weighted')))
print('\n auc:')
print('{:.2%}\n'.format(roc_auc_score(one_hot(test_labels), one_hot(y_pred), average='weighted', multi_class='ovo')))

Logistic regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as s
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_scoren

#Let's build a pipeline with only a scaler and a linear SVM
clf = LogisticRegression()
params = {'penalty': ['l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}

grid_search_lg = GridSearchCV(clf, param_grid = params,scoring = 'accuracy')

#Let's train and test the learning system
grid_search_lg.fit(train_features_pca, y_train)


#Here we just print the best accuracy and the corresponding values for the parameters
print("The best parameters are %s with an accuracy of %0.4f"%(grid_search_lg.best_params_, grid_search_lg.best_score_))


y_pred= grid_search_lg.predict(test_features_pca)
cm = confusion_matrix(y_pred, test_labels)

#  metrics 

print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(test_labels, y_pred)))
print('\nPrecision:')
print('{:.2%}\n'.format(precision_score(test_labels, y_pred,average='weighted')))
print('\nrecall:')
print('{:.2%}\n'.format(recall_score(test_labels, y_pred,average='weighted')))
print('\nf1 score:')
print('{:.2%}\n'.format(f1_score(test_labels, y_pred,average='weighted')))
print('\n auc:')
print('{:.2%}\n'.format(roc_auc_score(one_hot(test_labels), one_hot(y_pred), average='weighted', multi_class='ovo')))

Random Forest

In [ ]:
#Random forest:

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score,roc_curve, auc
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn


n_estimators = [200,500,1000,2000]
max_depth = [25,50,60]
min_samples_split = [10,15,25]
min_samples_leaf = [10,15,25] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

clf=RandomForestClassifier()

grid_search_rf = GridSearchCV(clf, param_grid = hyperF,scoring = 'accuracy')


#Let's train and test the learning system
grid_search_rf.fit(train_features_pca, y_train)


#Here we just print the best accuracy and the corresponding values for the parameters
print("The best parameters are %s with an accuracy of %0.4f"%(grid_search_rf.best_params_, grid_search_rf.best_score_))


y_pred= grid_search_rf.predict(test_features_pca)
cm = confusion_matrix(y_pred, test_labels)

#metrics 

print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(test_labels, y_pred)))
print('\nPrecision:')
print('{:.2%}\n'.format(precision_score(test_labels, y_pred,average='weighted')))
print('\nrecall:')
print('{:.2%}\n'.format(recall_score(test_labels, y_pred,average='weighted')))
print('\nf1 score:')
print('{:.2%}\n'.format(f1_score(test_labels, y_pred,average='weighted')))
print('\n auc:')
print('{:.2%}\n'.format(roc_auc_score(one_hot(test_labels), one_hot(y_pred), average='weighted', multi_class='ovo')))

Naive Baye's

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score


gnb = GaussianNB()
naive_classifier = gnb.fit(train_features_pca, y_train)

y_pred = naive_classifier.predict(test_features_pca)

print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(test_labels, y_pred)))
print('\nPrecision:') 
print('{:.2%}\n'.format(precision_score(test_labels, y_pred,average='weighted')))
print('\nrecall:')
print('{:.2%}\n'.format(recall_score(test_labels, y_pred,average='weighted')))
print('\nf1 score:')
print('{:.2%}\n'.format(f1_score(test_labels, y_pred,average='weighted')))
print('\n auc:')
print('{:.2%}\n'.format(roc_auc_score(one_hot(test_labels), one_hot(y_pred), average='weighted', multi_class='ovo')))